In [20]:
import matplotlib.pyplot as plt
import matplotlib
import argparse
from estimator_helper import state_estimator, get_img_process
from nerf_helper import load_intrinsic_params
from nerf.utils import *
from nav.math_utils import vec_to_rot_matrix, rot_matrix_to_vec
from nerf.provider import NeRFDataset
from nav.planner_helper import Planner
import os

In [31]:
class Config:
    def __init__(self):
        self.weight = None
        self.path = 'data/nerf_synthetic/custom_8'  # Set default path
        self.O = True
        self.test = False
        self.workspace = 'trial_nerf_custom_8'
        self.seed = 0

        # Training options
        self.iters = 30000
        self.lr = 1e-2
        self.ckpt = 'latest'
        self.num_rays = 4096
        self.cuda_ray = False
        self.max_steps = 1024
        self.num_steps = 512
        self.upsample_steps = 0
        self.update_extra_interval = 16
        self.max_ray_batch = 4096

        # Network backbone options
        self.fp16 = False
        self.ff = False
        self.tcnn = False

        # Dataset options
        self.color_space = 'srgb'
        self.preload = False
        self.bound = 5.0
        self.scale = 0.33
        self.offset = [0, 0, 0]
        self.dt_gamma = 0.02
        self.min_near = 0.2
        self.density_thresh = 10
        self.bg_radius = -1

        # GUI options
        self.gui = False
        self.W = 1920
        self.H = 1080
        self.radius = 5
        self.fovy = 50
        self.max_spp = 64

        # Experimental
        self.error_map = False
        self.clip_text = ''
        self.rand_pose = -1

        # Additional configurations based on conditional logic
        self.path = os.getcwd() + "/" + self.path
        self.workspace = os.getcwd() + "/" + self.workspace
        if self.O:
            self.fp16 = True
            self.cuda_ray = False
            self.preload = False


In [32]:
matplotlib.use('tkagg')
device = torch.device('cuda:0')
# Configuration

# file_path = os.path.dirname()
# Create a config instance
opt = Config()
if opt.ff:
    opt.fp16 = False
    assert opt.bg_radius <= 0, "background model is not implemented for --ff"
    # Specific import for ff
    from nerf.network_ff import NeRFNetwork
elif opt.tcnn:
    opt.fp16 = False
    assert opt.bg_radius <= 0, "background model is not implemented for --tcnn"
    # Specific import for tcnn
    from nerf.network_tcnn import NeRFNetwork
else:
    # Default import if neither ff nor tcnn
    from nerf.network import NeRFNetwork

In [33]:
seed_everything(opt.seed)

nerf_model = NeRFNetwork(
    encoding="hashgrid",
    bound=opt.bound,
    cuda_ray=opt.cuda_ray,
    density_scale=1,
    min_near=opt.min_near,
    density_thresh=opt.density_thresh,
    bg_radius=opt.bg_radius,
)

nerf_model.eval()
metrics = [PSNRMeter(),]
intrinsics, img_W, img_H = load_intrinsic_params(opt)
nerf_model = nerf_model.to(device)


In [34]:
ckpt_path = os.path.join(opt.workspace, 'checkpoints')
checkpoint = None
if checkpoint is None:
    checkpoint_list = sorted(glob.glob(f'{ckpt_path}/ngp_ep*.pth'))
if checkpoint_list:
    checkpoint = checkpoint_list[-1]
    print(f"[INFO] Latest checkpoint is {checkpoint}")
else:
    print("[WARN] No checkpoint found, model randomly initialized.")

checkpoint_dict = torch.load(checkpoint, map_location='cpu')
if 'model' not in checkpoint_dict:
    nerf_model.load_state_dict(checkpoint_dict)
    print("[INFO] loaded model.")

else:
    missing_keys, unexpected_keys = nerf_model.load_state_dict(checkpoint_dict['model'], strict=False)
    print("[INFO] loaded model.")
    if len(missing_keys) > 0:
        print(f"[WARN] missing keys: {missing_keys}")
    if len(unexpected_keys) > 0:
        print(f"[WARN] unexpected keys: {unexpected_keys}")

[INFO] Latest checkpoint is /home/oem/workspace/raibo_arm/raisimGymTorch/env/envs/rsg_raibo_rough_terrain/trial_nerf_custom_8/checkpoints/ngp_ep0001.pth
[INFO] loaded model.
[WARN] unexpected keys: ['density_grid', 'density_bitfield', 'step_counter']


In [35]:
rot = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]], device=device, dtype=torch.float32)
density_fn = lambda x: nerf_model.density(x.reshape((-1, 3)) @ rot)['sigma'].reshape(x.shape[:-1])
render_fn = lambda rays_o, rays_d: nerf_model.render(rays_o, rays_d, staged=True, bg_color=1., perturb=False, **vars(opt))
get_rays_fn = lambda pose: get_rays(pose, intrinsics, img_H, img_W)

In [41]:
new_pose = torch.eye(4)
new_pose[:3, :3] = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]], device=device, dtype=torch.float32)
new_pose[:3, 3] = torch.tensor([0., 0., 0.5], device=device, dtype=torch.float32)
new_pose = new_pose.to(device)

rays = get_rays_fn(new_pose.reshape((1,4,4)))

render_fn(rays["rays_o"].reshape((img_H, img_W, -1)), rays["rays_d"].reshape((img_H, img_W, -1)))

OutOfMemoryError: CUDA out of memory. Tried to allocate 34.00 MiB (GPU 0; 7.78 GiB total capacity; 4.36 GiB already allocated; 114.19 MiB free; 4.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF